# An Analysis of Zodiac and Excellence in Career Paths


This project seeks to To accomplish this, I am reading several different datasets into pandas dataframes, cleaning them, loading them into a SQLite database and combining them with a SQL join, before analyzing the results.  

## Birthdates Data Set
### Reading Txt File

I began by reading the first dataset, famous-birthdates.txt local machine into a pandas dataframe, starting with [this]https://github.com/richard512/Little-Big-Data/blob/master/famous-birthdates.csv birthdate file courtesy of fellow GitHub-er richard512. 

***To run this code on your own, you may need to replace the file path below with the filepath at which you cloned the repo on your local machine. ***


In [1]:
import pandas as pd
import numpy 

#Read in txt file into pandas df
bd_df = pd.read_csv(r'Data\famous-birthdates.txt', delimiter = " ")
bd_df.head()

,name,lastname,firstname,articleNum,birthDate,birthMonth,birthDay,zodiac
1,Aaliyah,Aaliyah,NaN,0,1979-01-16,1.0,16.0,Capricorn
2,"Aaron, Hank",Aaron,Hank,46,1934-02-05,2.0,5.0,Aquarius
3,"Abacha, Sani",Abacha,Sani,2,1943-09-20,9.0,20.0,Virgo
4,"Abbado, Claudio",Abbado,Claudio,9,1933-06-26,6.0,26.0,Cancer
5,"Abbas, Mahmoud",Abbas,Mahmoud,306,1935-03-26,3.0,26.0,Aries


### Cleaning Birthdate Data Set
#### Working with Birthdates

Since this dataset is the main source of birthdates for my project, I am only interested in keeping rows with a birthdate. Given that birthdates are static, I assigned each a Date ID based on the day of the year to simplify this dataset and better prepare it for a SQL table. First, however, I converted the Birthdate column to datetime, before again dropping any rows with a missing birthdate.   

Before removing the birthdate altogether, however, I extracted the year. This year will be combined later with a portion of the person's first and last name to create a unique ID on which to join my datasets. 

In [2]:
#number of rows before any cleaning
len(bd_df)

4710

In [3]:
#drop rows without a birthDate
bd_df.dropna(subset = ['birthDate'], inplace=True)
len(bd_df)

4491

In [4]:
#convert birthdate to date/time data type, 
bd_df['birthDate'] = pd.to_datetime(bd_df['birthDate'], errors='coerce')
#then convert to a day of the year
bd_df['Date_Id'] = bd_df['birthDate'].dt.dayofyear
#drop rows where the date_id is NULL 
bd_df.dropna(subset = ['Date_Id'], inplace=True)
#extract birthyear from birthdate
bd_df['year'] = bd_df['birthDate'].dt.year
#drop unnecessary columns
bd_df = bd_df.drop(columns=['articleNum', 'birthDate', 'birthMonth', 'birthDay', 'zodiac'])
bd_df.head()


,name,lastname,firstname,Date_Id,year
1,Aaliyah,Aaliyah,NaN,16.0,1979
2,"Aaron, Hank",Aaron,Hank,36.0,1934
3,"Abacha, Sani",Abacha,Sani,263.0,1943
4,"Abbado, Claudio",Abbado,Claudio,177.0,1933
5,"Abbas, Mahmoud",Abbas,Mahmoud,85.0,1935


#### Cleaning Names

Luckily, this dataset separated out first and last name, but there were a couple of steps I took to further standardize the formatting of names. 

1) In instances where a person goes by only one name, I moved that name from lastname to firstname, and set the lastname to NULL
2) There are a handful of instances where the value in lastname is not exactly a lastname, (e.g. Abdullah II, King of Jordan). I replaced those manually.
3) In preparation of my next step, I removed any spaces from first and last names.
4) Finally, any instance where a lastname is NULL was set to be blank.

In [5]:
#fixing instances where the person goes by a single name 
# Copy 'LastName' to 'FirstName' where 'FirstName' is null
one_name = bd_df['firstname'].isnull()
bd_df.loc[one_name, 'firstname'] = bd_df.loc[one_name, 'lastname']
# Set 'LastName' to null for the rows where 'FirstName' was null
bd_df.loc[one_name, 'lastname'] = pd.NA
#fix instances where last name is not really a last name
bd_df.at[9, 'lastname'] = ''
bd_df.at[9, 'firstname'] = 'Abdullah II, King of Jordan'
bd_df.at[1212, 'lastname'] = ''
bd_df.at[1212, 'firstname'] = 'Elizabeth II, Queen of Great Britain'
bd_df.at[3036, 'lastname'] = ''
bd_df.at[3036, 'firstname'] = 'Nicholas II, Czar of Russia'
#Remove Spaces in first and lastname
bd_df['firstname2'] = bd_df['firstname'].str.replace(' ', '')
bd_df['lastname2'] = bd_df['lastname'].str.replace(' ', '')
#fill NA with blanks
bd_df = bd_df.fillna('')
bd_df.head()

,name,lastname,firstname,Date_Id,year,firstname2,lastname2
1,Aaliyah,,Aaliyah,16.0,1979,Aaliyah,
2,"Aaron, Hank",Aaron,Hank,36.0,1934,Hank,Aaron
3,"Abacha, Sani",Abacha,Sani,263.0,1943,Sani,Abacha
4,"Abbado, Claudio",Abbado,Claudio,177.0,1933,Claudio,Abbado
5,"Abbas, Mahmoud",Abbas,Mahmoud,85.0,1935,Mahmoud,Abbas


### Creating Joinable Value

I also needed a way to join my two sets of names.  Since they don't follow the same naming conventions, I decided to create a new column that takes the first 3 letters of a person's first name and last name and their birthyear and concatenates them to a (reasonably) unique value.  There were a very few duplicates that I handled manually for now.  

Finally, I dropped the unneeded columns and cleaned up headings before loading this into a SQLite database.  

In [6]:
#get the first 3 letters of the first and last name
#cast year to a string and concatenate to create a value to join with other dataset
bd_df['First3'] = bd_df['firstname2'].str[:3]
bd_df['Last3'] = bd_df['lastname2'].str[:3]
bd_df['year'] = bd_df['year'].astype(str)
bd_df['People_Lookup'] = bd_df['First3'] + bd_df['Last3'] + bd_df['year']
#fix some duplicate people lookups
bd_df.at[264, 'People_Lookup'] = 'KatBatt1948'
bd_df.at[2410, 'People_Lookup'] = 'LeeK1923'
bd_df.at[2485, 'People_Lookup'] = 'LiK1928'
#clean-up headings, drop added columns
bd_df = bd_df.rename(columns={'name': 'ImportName', 'lastname': 'LastName','firstname': 'FirstName', 'Date_Id': 'DateID'})
bd_df = bd_df.drop(columns=['year', 'First3', 'Last3', 'firstname2', 'lastname2'])
bd_df.head()


,ImportName,LastName,FirstName,DateID,People_Lookup
1,Aaliyah,,Aaliyah,16.0,Aal1979
2,"Aaron, Hank",Aaron,Hank,36.0,HanAar1934
3,"Abacha, Sani",Abacha,Sani,263.0,SanAba1943
4,"Abbado, Claudio",Abbado,Claudio,177.0,ClaAbb1933
5,"Abbas, Mahmoud",Abbas,Mahmoud,85.0,MahAbb1935


In [7]:
#write dataframe to table
import sqlite3
connection = sqlite3.connect('Zodiac_Analysis.db')
bd_df.to_sql('Famous_People_Import', connection, if_exists='replace')

4477

## Pantheon People Dataset
###Reading Pantheon Data
Next, I imported and cleaned the data from Harvard's Pantheon data project.  

***To run this code on your own, you may need to replace the file path below with the filepath at which you cloned the repo on your local machine. ***

In [8]:
#Pull csv into pandas dataframe
panth_data = pd.read_csv(r'Data\pantheon_people.csv', delimiter = ",")
len(panth_data)

11341

### Cleaning Pantheon Data
In cleaning this dataset, I first dropped the unnecessary columns. I then began cleaning up the birthyear column, as it will use this value to join with the birthdate date set. 

There were also a few irregular values in the birthyear column, which I forced to a numeric data type and dropped any rows without a birthyear. I also chose to drop any rows with a birthdate before 1500, partially because my other dataset features mostly modern individuals, but also because birthdates prior to that time are probably unreliable. Finally, I converted the year to an integer.

In [9]:
#drop unneeded columns
panth_data = panth_data.drop(columns=['en_curid','numlangs','countryCode','LAT','LON', 'TotalPageViews', 'L_star', 'StdDevPageViews', 'PageViewsEnglish', 'PageViewsNonEnglish', 'AverageViews' , 'HPI'                 ]) 
#fix some bad values in the birthyear column
badyr = panth_data['birthyear'].str.extract(r'^(\d{4})', expand=False)
panth_data['birthyear'] = pd.to_numeric(badyr)
#Drop na's in year
panth_data.dropna(subset=["birthyear"], inplace=True)
#drop where year is less than 1500 - prob won't have an accurate birthdate
panth_data = panth_data.drop(panth_data[panth_data['birthyear'] < 1500].index)
#convert birth year to int
panth_data['birthyear'] = panth_data['birthyear'].astype('int64')
panth_data.head(10)

,name,birthcity,birthstate,countryName,countryCode3,continentName,birthyear,gender,occupation,industry,domain
0,Abraham Lincoln,Hodgenville,KY,UNITED STATES,USA,North America,1809,Male,POLITICIAN,GOVERNMENT,INSTITUTIONS
2,Ayn Rand,Saint Petersburg,NaN,Russia,RUS,Europe,1905,Female,WRITER,LANGUAGE,HUMANITIES
3,Andre Agassi,Las Vegas,NV,UNITED STATES,USA,North America,1970,Male,TENNIS PLAYER,INDIVIDUAL SPORTS,SPORTS
4,Aldous Huxley,Godalming,NaN,UNITED KINGDOM,GBR,Europe,1894,Male,WRITER,LANGUAGE,HUMANITIES
5,Andrei Tarkovsky,Zavrazhye,NaN,Russia,RUS,Europe,1932,Male,FILM DIRECTOR,FILM AND THEATRE,ARTS
6,Arthur Schopenhauer,Gdańsk,NaN,POLAND,POL,Europe,1788,Male,PHILOSOPHER,PHILOSOPHY,HUMANITIES
7,Albert Einstein,Ulm,NaN,Germany,DEU,Europe,1879,Male,PHYSICIST,NATURAL SCIENCES,SCIENCE & TECHNOLOGY
9,Alfred Hitchcock,Leytonstone,NaN,UNITED KINGDOM,GBR,Europe,1899,Male,FILM DIRECTOR,FILM AND THEATRE,ARTS
11,Alfred Nobel,Stockholm,NaN,SWEDEN,SWE,Europe,1833,Male,CHEMIST,NATURAL SCIENCES,SCIENCE & TECHNOLOGY
12,Alexander Graham Bell,Edinburgh,NaN,UNITED KINGDOM,GBR,Europe,1847,Male,INVENTOR,INVENTION,SCIENCE & TECHNOLOGY


In [10]:
#determine the number of words in each name field
panth_data["WordsCt"] = panth_data["name"].apply(lambda n: len(n.split()))
def split_names(row):
    if row['WordsCt'] == 2:
        first, last = row['name'].split()[:2]
        return pd.Series({'firstname': first, 'middlename': '', 'lastname': last})
    elif row['WordsCt'] == 3:
        first, middle, last = row['name'].split()[:3]
        return pd.Series({'firstname': first, 'middlename': middle, 'lastname': last})
    elif row['WordsCt'] == 1:
        first = row['name'].split()[:1]
        return pd.Series({'firstname': row['name'], 'middlename': '', 'lastname': ''})        
    else:
        return pd.Series({'firstname': None, 'middlename': None, 'lastname': None})
#Change NA to blanks
    panth_data = panth_data.fillna('')
# Apply the split_names function 
panth_data = pd.concat([panth_data, panth_data.apply(split_names, axis=1)], axis=1)
#drop rows that could not be split out
panth_data = panth_data.dropna(subset=['firstname'])
panth_data.head()

,name,birthcity,birthstate,countryName,countryCode3,continentName,birthyear,gender,occupation,industry,domain,WordsCt,firstname,middlename,lastname
0,Abraham Lincoln,Hodgenville,KY,UNITED STATES,USA,North America,1809,Male,POLITICIAN,GOVERNMENT,INSTITUTIONS,2,Abraham,,Lincoln
2,Ayn Rand,Saint Petersburg,NaN,Russia,RUS,Europe,1905,Female,WRITER,LANGUAGE,HUMANITIES,2,Ayn,,Rand
3,Andre Agassi,Las Vegas,NV,UNITED STATES,USA,North America,1970,Male,TENNIS PLAYER,INDIVIDUAL SPORTS,SPORTS,2,Andre,,Agassi
4,Aldous Huxley,Godalming,NaN,UNITED KINGDOM,GBR,Europe,1894,Male,WRITER,LANGUAGE,HUMANITIES,2,Aldous,,Huxley
5,Andrei Tarkovsky,Zavrazhye,NaN,Russia,RUS,Europe,1932,Male,FILM DIRECTOR,FILM AND THEATRE,ARTS,2,Andrei,,Tarkovsky


### Creating a Joinable Value

Now that I have separate first and last names, in preparation of creating a lookup field, I took a number of steps to standardize the data. 
1) I removed spaces and punctuation from first, middle, and lastname. 
2) There were also a number of people who go by their first and/or middle initial (AA Milne, for example), so I created a function to correct these instances.
3) Removed non-English characters and replaced them with their Engish equivalent.
3) Fixed some other one-off names manually. 
4) As I did above, I split out the first 3 letters of the first and last name and combined it with the birthyear, and then  dropped the unneeded columns. 
5) I tidyed up some unnecessary columns
6) Corrected capitalization 
7) replaced NA's with blanks 

Finally, wrote the data into a SQL table. 

In [11]:
#Remove Spaces 
panth_data['firstname'] = panth_data['firstname'].str.replace(' ', '')
panth_data['lastname'] = panth_data['lastname'].str.replace(' ', '')
panth_data['middlename'] = panth_data['middlename'].str.replace(' ', '')
#Remove punctuation 
punctuation = r'[^\w\s]'
panth_data['firstname'] = panth_data['firstname'].replace(punctuation, '', regex=True)
panth_data['lastname'] = panth_data['lastname'].replace(punctuation, '', regex=True)
panth_data['middlename'] = panth_data['middlename'].replace(punctuation, '', regex=True)
#Fixed names that go by first and middle initial 
def first_initial(row):
    if len(row['firstname']) == 1:
        return row['firstname'] + row['middlename']
    else:
        return row['firstname']
# Apply the function 
panth_data['firstname'] = panth_data['firstname'].astype('str')
panth_data['middlename'] = panth_data['middlename'].astype('str')
panth_data['firstname'] = panth_data.apply(first_initial, axis=1)
#get the first 3 letters of the first and last name
panth_data['First3'] = panth_data['firstname'].str[:3]
panth_data['Last3'] = panth_data['lastname'].str[:3]

#cast year to a string and concatenate to create a value to join with other dataset
panth_data['birthyear2'] = panth_data['birthyear'].astype(str)
panth_data['People_Lookup'] = panth_data['First3'] + panth_data['Last3'] + panth_data['birthyear2']
panth_data = panth_data.drop(columns=['WordsCt', 'First3', 'Last3', 'birthyear2', 'birthyear'])

panth_data.head()

,name,birthcity,birthstate,countryName,countryCode3,continentName,gender,occupation,industry,domain,firstname,middlename,lastname,People_Lookup
0,Abraham Lincoln,Hodgenville,KY,UNITED STATES,USA,North America,Male,POLITICIAN,GOVERNMENT,INSTITUTIONS,Abraham,,Lincoln,AbrLin1809
2,Ayn Rand,Saint Petersburg,NaN,Russia,RUS,Europe,Female,WRITER,LANGUAGE,HUMANITIES,Ayn,,Rand,AynRan1905
3,Andre Agassi,Las Vegas,NV,UNITED STATES,USA,North America,Male,TENNIS PLAYER,INDIVIDUAL SPORTS,SPORTS,Andre,,Agassi,AndAga1970
4,Aldous Huxley,Godalming,NaN,UNITED KINGDOM,GBR,Europe,Male,WRITER,LANGUAGE,HUMANITIES,Aldous,,Huxley,AldHux1894
5,Andrei Tarkovsky,Zavrazhye,NaN,Russia,RUS,Europe,Male,FILM DIRECTOR,FILM AND THEATRE,ARTS,Andrei,,Tarkovsky,AndTar1932


In [12]:
#fix some one-off values
panth_data.at[51, 'lastname'] = ''
panth_data.at[51, 'firstname'] = 'Ahmed I'
panth_data.at[52, 'lastname'] = ''
panth_data.at[52, 'firstname'] = 'Ahmed II'
panth_data.at[53, 'lastname'] = ''
panth_data.at[53, 'firstname'] = 'Ahmed III'
panth_data.at[159, 'lastname'] = ''
panth_data.at[159, 'firstname'] = 'Abdul Hamid I'
panth_data.at[650, 'lastname'] = ''
panth_data.at[650, 'firstname'] = 'Ice T'
#fix capitalization
panth_data['domain'] = panth_data['domain'].str.title()
panth_data['industry'] = panth_data['industry'].str.title()
panth_data['occupation'] = panth_data['occupation'].str.title()
#rename headings in prep of db import
panth_data = panth_data.rename(columns={'domain': 'Area', 'countryCode3': 'Country_Code'})
#replaced NA's with blanks
panth_data = panth_data.fillna('')
#Fixed non-English characters
from unidecode import unidecode 
def remove_non_english(text):
    return unidecode(text)
panth_data['People_Lookup'] = panth_data['People_Lookup'].apply(remove_non_english)
panth_data = panth_data.fillna('')
#write to SQL
panth_data.to_sql('Pantheon_People_Import', connection, if_exists='replace')

8867

## Import Zodiac file and load into SQL
Next I simply imported the CSV Zodiac file, set the index, and loaded it into SQL. 

***To run this code on your own, you may need to replace the file path below with the filepath at which you cloned the repo on your local machine. ***

In [13]:
#Pull csv into pandas dataframe
zod_data = pd.read_csv(r'Data\Zodiac.csv', delimiter = ",")
#set DateID to index
zod_data = zod_data.set_index('Date_ID')
zod_data.head()

,Month,Day,Zodiac,Element,Career Strengths
Date_ID,,,,,
1,1,1,Capricorn,Earth,"High-Achieving, Driven"
2,1,2,Capricorn,Earth,"High-Achieving, Driven"
3,1,3,Capricorn,Earth,"High-Achieving, Driven"
4,1,4,Capricorn,Earth,"High-Achieving, Driven"
5,1,5,Capricorn,Earth,"High-Achieving, Driven"


In [14]:
#write to SQL
zod_data.to_sql('Zodiac', connection, if_exists='replace')

366

## Separate area, industry, and occupation into their own tables.
To eliminate redundancy in my database and simplify later calculations, I split the area, industry, and occupation columns from the Pantheon data and loaded them to their own tables. For each, I added an ID, reversed the order of the columns, and made it the index value before loading into a SQL table.

In [15]:
#Read in SQL query
industrydf = pd.read_sql_query("select distinct industry from pantheon_people_import;", connection)
#add Industry ID
industrydf['Industry_ID'] = range(1, 28 , 1)
#Reverse column order and make domain_id the index
industrydf.iloc[:,[1,0]]
industrydf = industrydf.set_index('Industry_ID')
industrydf.head()

,industry
Industry_ID,
1,Government
2,Language
3,Individual Sports
4,Film And Theatre
5,Philosophy


In [16]:
#Read in SQL query
areadf = pd.read_sql_query("select distinct area from pantheon_people_import;", connection)
#add Area ID to each 
areadf['Area_Id'] = range(1, 9 , 1)
#Reverse column order and make domain_id the index
areadf.iloc[:,[1,0]]
areadf = areadf.set_index('Area_Id')
areadf.head()

,Area
Area_Id,
1,Institutions
2,Humanities
3,Sports
4,Arts
5,Science & Technology


In [17]:
#Read in SQL query
occdf = pd.read_sql_query("select distinct occupation from pantheon_people_import;", connection)
#add Industry ID to each 
occdf['Occupation_ID'] = range(1, 89 , 1)
#Reverse column order and make domain_id the index
occdf.iloc[:,[1,0]]
occdf = occdf.set_index('Occupation_ID')
occdf.head()

,occupation
Occupation_ID,
1,Politician
2,Writer
3,Tennis Player
4,Film Director
5,Philosopher


In [18]:
industrydf.to_sql('Industry', connection, if_exists='replace')
occdf.to_sql('Occupation', connection, if_exists='replace')
areadf.to_sql('Area', connection, if_exists='replace')

8

## Joining Datasets

I then used the previously worked data to create a People table. Using a union, I brought in all unique PeopleLookup ID's, then added this to a new SQL table to begin forming People. 

Next, I pulled data in from the previously-created tables to combine all my data, cleaned it:
    1)dropped the people lookup
    2)adding a person_id and set it to be the index
    3)filling in names as needed
    4)fixing NaN's and nones
    5)fixed capitalization 
    6)set ID values to integers

Finally, I wrote it back into the people table.  

In [19]:
# Read sqlite query results into a pandas DataFrame
query=""" select fpi.People_Lookup  
    from famous_people_import fpi
    union
    select ppi.people_lookup
    from pantheon_people_import ppi"""
peopledf = pd.read_sql_query(query, connection)
peopledf.head(10)  

,People_Lookup
0,50Cen1975
1,AAMil1882
2,AAlTau1924
3,AEHot1920
4,AJAye1910
5,AJCro1896
6,ANWil1950
7,AOSco1966
8,ARGur1930
9,ARRah1966


In [20]:
#write to SQL table
peopledf.to_sql('People', connection, if_exists='replace')

12061

In [21]:
#pull in all people data
query=""" select p.People_Lookup, ppi.name, fpi.firstname, fpi.lastname, ppi.countryName, ppi.gender
        ,a.Area_Id ,i.Industry_ID ,o.Occupation_ID ,fpi.DateID   
from people p
left join famous_people_import fpi on fpi.People_Lookup=p.People_Lookup
left join pantheon_people_import ppi on ppi.People_Lookup=p.People_Lookup
left join area a on a.Area=ppi.Area
left join industry i on i.industry=ppi.industry
left join occupation o on o.occupation=ppi.occupation"""
peopledf = pd.read_sql_query(query, connection)
peopledf.head()

,People_Lookup,name,FirstName,LastName,countryName,gender,Area_Id,Industry_ID,Occupation_ID,DateID
0,50Cen1975,50 Cent,None,None,United States,Male,4.0,14.0,27.0,NaN
1,AAMil1882,A. A. Milne,A A,Milne,UNITED KINGDOM,Male,2.0,2.0,2.0,18.0
2,AAlTau1924,None,A Alfred,Taubman,None,None,NaN,NaN,NaN,31.0
3,AEHot1920,None,A E,Hotchner,None,None,NaN,NaN,NaN,180.0
4,AJAye1910,A.J. Ayer,None,None,United Kingdom,Male,2.0,5.0,5.0,NaN


In [22]:
#Drop PeopleLookup column
peopledf = peopledf.drop(columns=['People_Lookup'])
#add PersonID to each 
peopledf['Person_ID'] = range(1, len(peopledf)+1, 1)
#Set Person_ID to the index
peopledf = peopledf.set_index('Person_ID')
#Fixed Capitalization in County
peopledf['countryName'] = peopledf['countryName'].str.title()
peopledf.head()

,name,FirstName,LastName,countryName,gender,Area_Id,Industry_ID,Occupation_ID,DateID
Person_ID,,,,,,,,,
1,50 Cent,None,None,United States,Male,4.0,14.0,27.0,NaN
2,A. A. Milne,A A,Milne,United Kingdom,Male,2.0,2.0,2.0,18.0
3,None,A Alfred,Taubman,None,None,NaN,NaN,NaN,31.0
4,None,A E,Hotchner,None,None,NaN,NaN,NaN,180.0
5,A.J. Ayer,None,None,United Kingdom,Male,2.0,5.0,5.0,NaN


In [23]:
#for people with no value in name, combine first name and last name
def fill_name(row):
    if pd.isnull(row['name']):
        return row['FirstName'] + ' ' + row['LastName']
    else:
        return row['name']
# Apply the function to the DataFrame
peopledf['name'] = peopledf.apply(fill_name, axis=1)
#CHange ID fields to ints
peopledf['Industry_ID'] = pd.to_numeric(peopledf['Industry_ID'], errors='coerce')
peopledf['Industry_ID'] = peopledf['Industry_ID'].fillna(0).astype('int64')
peopledf['Occupation_ID'] = pd.to_numeric(peopledf['Occupation_ID'], errors='coerce')
peopledf['Occupation_ID'] = peopledf['Occupation_ID'].fillna(0).astype('int64')
peopledf['Area_Id'] = pd.to_numeric(peopledf['Area_Id'], errors='coerce')
peopledf['Area_Id'] = peopledf['Area_Id'].fillna(0).astype('int64')
peopledf['DateID'] = pd.to_numeric(peopledf['DateID'], errors='coerce')
peopledf['DateID'] = peopledf['DateID'].fillna(0).astype('int64')
#fill na with blank
peopledf = peopledf.fillna('')
#drop first name and last name
peopledf = peopledf.drop(columns=['FirstName', 'LastName'])
peopledf.head()

,name,countryName,gender,Area_Id,Industry_ID,Occupation_ID,DateID
Person_ID,,,,,,,
1,50 Cent,United States,Male,4,14,27,0
2,A. A. Milne,United Kingdom,Male,2,2,2,18
3,A Alfred Taubman,,,0,0,0,31
4,A E Hotchner,,,0,0,0,180
5,A.J. Ayer,United Kingdom,Male,2,5,5,0


In [24]:
#write to SQL table
peopledf.to_sql('People', connection, if_exists='replace')

12065

## Supplemental Data
### Rows with no area, industry, occupation ID
Outside of this notebook, I created a csv that filled in area, industry, and occupation ID's for some of the people in my birthdates data set that did not appear in the Pantheon data.  I read this data into a dataframe and then performed a SQL update to add the data to the People table in my database.

***To run this code on your own, you may need to replace the file path below with the filepath to which you cloned the repo on your local machine. ***


In [25]:
#Read in First Supplemental CSV
missingareadf = pd.read_csv(r'Data\Blank_Pantheon.csv', delimiter = ",")
#set Person_Id to the index
missingareadf = missingareadf.set_index('Person_ID')
missingareadf.head()

,name,Country,gender,Area_ID,Industry_ID,Occupation_ID,DateID
Person_ID,,,,,,,
16,Aaron Boone,United States,male,3.0,21.0,32.0,68
21,Aaron Glenn,United States,male,3.0,3.0,42.0,198
23,Aaron Jay Kernis,United States,male,4.0,14.0,16.0,15
34,Abbey Lincoln,United States,female,4.0,14.0,27.0,218
32,Abby Joseph Cohen,United States,female,5.0,16.0,20.0,1


In [26]:
# Write DataFrame to a temporary table
missingareadf.to_sql('temp_table1', connection, if_exists='replace')
 
# Create a SQL UPDATE statement
update_sql = """
UPDATE people
SET countryName = temp_table1.Country,
    gender = temp_table1.gender,
    Area_Id = temp_table1.Area_ID,
    Industry_ID = temp_table1.Industry_ID,
    Occupation_ID = temp_table1.Occupation_ID
FROM temp_table1
WHERE people.Person_ID = temp_table1.Person_ID
"""
#Execute the SQL update
cursor=connection.cursor()
cursor.execute(update_sql)
#Commit the changes
connection.commit()

### Rows with no birthdate
Outside of this notebook, I created a supplemental dataset to fill in the people for whom we did not have a birthdate.  I read this data into a dataframe and then performed a SQL update to add the data to the People table in my database.

***To run this code on your own, you may need to replace the file path below with the filepath at which you cloned the repo on your local machine. ***


In [27]:
#Read in Supplemental CSV
missingbdaydf = pd.read_csv(r'Data\Missingbirthdate.csv', delimiter = ",")
#set Person_Id to the index
missingbdaydf = missingbdaydf.set_index('Person_ID')
missingbdaydf.head()

,Day_ID
Person_ID,
6309,1
9671,1
10257,1
1998,1
2057,1


In [28]:
# Write DataFrame to a temporary table
missingbdaydf.to_sql('temp_table2', connection, if_exists='replace')
 
# Create a SQL UPDATE statement
update_sql = """
UPDATE people
SET DateID = temp_table2.Day_ID
FROM temp_table2
WHERE people.Person_ID = temp_table2.Person_ID
"""
#Execute the SQL update
cursor=connection.cursor()
cursor.execute(update_sql)
#Commit the changes
connection.commit()

## Produce Final DataSet

Finally, I produced a final, polished dataset on which to perform analysis and produce a Tableau dashboard. 
The analysis can be found here:    ADD LINK HERE

In [29]:
# Read sqlite query results into a pandas DataFrame
query=""" select p.name, p.gender, p.countryName
   , a.area, i.industry , o.occupation
   ,z.Zodiac ,z.Element   
from people p 
    inner join area a on a.Area_Id=p.Area_Id
    inner join industry i on i.industry_id=p.Industry_ID
    inner join occupation o on o.occupation_id=p.Occupation_ID
    inner join zodiac z on p.dateID=z.Date_ID
"""
finaldf = pd.read_sql_query(query, connection)
finaldf.head(25)  

,name,gender,countryName,Area,industry,occupation,Zodiac,Element
0,A. A. Milne,Male,United Kingdom,Humanities,Language,Writer,Capricorn,Earth
1,Aaliyah,Female,United States,Arts,Music,Singer,Capricorn,Earth
2,Aaron Boone,male,United States,Sports,Team Sports,Baseball Player,Pisces,Water
3,Aaron Burr,Male,United States,Institutions,Government,Politician,Aquarius,Air
4,Aaron Copland,Male,United States,Arts,Music,Composer,Scorpio,Water
5,Aaron Glenn,male,United States,Sports,Individual Sports,Athlete,Cancer,Water
6,Aaron Jay Kernis,male,United States,Arts,Music,Composer,Capricorn,Earth
7,Aaron Sorkin,Male,United States,Humanities,Language,Writer,Gemini,Air
8,Abby Joseph Cohen,female,United States,Science & Technology,Social Sciences,Economist,Capricorn,Earth
9,Abbas Kiarostami,Male,Iran,Arts,Film And Theatre,Film Director,Cancer,Water


In [30]:
#write to CSV
finaldf.to_csv('Zodiac_Analysis.csv')